In [1]:
import sys
!{sys.executable} -m pip install --upgrade "pyautogen>=0.2.20" "autogen-ext[mcp,openai,azure]" git+https://github.com/adhikasp/mcp-weather.git uv autogen --quiet


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.2/859.2 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.2 MB

In [2]:
import asyncio
import nest_asyncio
import requests
import os
from datetime import date, datetime
from typing import Dict, List, Optional

from autogen import AssistantAgent, UserProxyAgent
from pydantic import BaseModel, Field

# Apply nest_asyncio to allow running asyncio in a notebook environment
nest_asyncio.apply()
import asyncio

# Prevent nested run_until_complete from breaking
_orig_run_until_complete = asyncio.BaseEventLoop.run_until_complete

def _patched_run_until_complete(self, future):
    if getattr(self, "_is_running", False):
        # If loop is already running, just await instead of re-entering
        return future.send(None)
    return _orig_run_until_complete(self, future)

asyncio.BaseEventLoop.run_until_complete = _patched_run_until_complete

# --- Pydantic Models for Structured Data ---
# These models define the structure of the data RETURNED by your tool.
class DailyForecast(BaseModel):
    date: date
    temp_min: float = Field(..., description="The minimum forecasted temperature for the day.")
    temp_max: float = Field(..., description="The maximum forecasted temperature for the day.")
    condition: str = Field(..., description="A brief description of the forecasted weather conditions.")

class CurrentConditions(BaseModel):
    temp: float
    wind_speed: float
    condition: str
    alerts: List[str] = Field(default_factory=list, description="A list of active weather alerts.")

class CityWeather(BaseModel):
    city: str
    state: str
    current_conditions: CurrentConditions
    forecast: List[DailyForecast] = Field(..., description="A list of daily forecasts for the next 5 days.")

from google.colab import userdata
os.environ["OPENWEATHER_API_KEY"] = userdata.get('OPENWEATHER_API_KEY')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_BASE_URL"] = userdata.get('AZURE_OPENAI_BASE_URL')


# --- Helper Functions (used by the tool) ---
def _get_coordinates_for_city(city: str, state: str, api_key: str) -> dict:
    """Fetches the latitude, longitude, and full name for a given city and state."""
    query = f"{city},{state},US"
    geocode_url = f"http://api.openweathermap.org/geo/1.0/direct?q={query}&limit=1&appid={api_key}"
    response = requests.get(geocode_url)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise ValueError(f"Location '{city}, {state}' not found.")

    location_data = data[0]
    return {
        "lat": location_data["lat"],
        "lon": location_data["lon"],
        "full_name": f"{location_data.get('name', city)}, {location_data.get('state', state)}"
    }

# --- Pydantic-Powered Weather Tool Class ---
class PydanticWeatherTool:
    def __init__(self, api_key: str):
        if not api_key or api_key == "YOUR_OPENWEATHER_API_KEY":
            raise ValueError("OpenWeatherMap API key is missing. Please set the OPENWEATHER_API_KEY variable.")
        self.api_key = api_key

    def get_weather_for_cities(self, locations: List[Dict[str, str]]) -> List[dict]:
        """
        Takes a list of city/state dictionaries, fetches detailed weather data,
        and returns it as a list of JSON objects conforming to the CityWeather Pydantic model.
        """
        weather_reports = []
        for loc in locations:
            try:
                coords = _get_coordinates_for_city(city=loc['city'], state=loc['state'], api_key=self.api_key)
                one_call_url = f"https://api.openweathermap.org/data/3.0/onecall?lat={coords['lat']}&lon={coords['lon']}&exclude=minutely,hourly&appid={self.api_key}&units=imperial"
                response = requests.get(one_call_url)
                response.raise_for_status()
                weather_data = response.json()

                current = CurrentConditions(
                    temp=weather_data['current']['temp'],
                    wind_speed=weather_data['current']['wind_speed'],
                    condition=weather_data['current']['weather'][0]['description'],
                    alerts=[alert['event'] for alert in weather_data.get('alerts', [])]
                )

                forecast = [
                    DailyForecast(
                        date=datetime.fromtimestamp(day['dt']).date(),
                        temp_min=day['temp']['min'],
                        temp_max=day['temp']['max'],
                        condition=day['weather'][0]['description']
                    ) for day in weather_data['daily'][:5]
                ]

                city_weather = CityWeather(
                    city=loc['city'],
                    state=loc['state'],
                    current_conditions=current,
                    forecast=forecast
                )
                weather_reports.append(city_weather.model_dump(mode='json'))

            except Exception as e:
                weather_reports.append({"city": loc['city'], "state": loc['state'], "error": str(e)})

        return weather_reports

# execution Logic -------------------
async def run_detailed_weather_report():
    """Sets up and runs the weather reporting agent and returns the result."""

    weather_tool = PydanticWeatherTool(api_key=os.environ["OPENWEATHER_API_KEY"] )

    llm_config = {
        "model": "gpt-4p1-mini-biopro",
        "api_type": "azure",
        "api_key": os.environ["AZURE_OPENAI_API_KEY"],
        "base_url": os.environ["AZURE_OPENAI_BASE_URL"],
        "api_version": "2024-12-01-preview",
        "max_tokens": 8000,
    }

    correct_tool_definition = {
        "type": "function",
        "function": {
            "name": "get_weather_for_cities",
            "description": "Fetches current conditions and a 5-day forecast for a list of city/state locations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "locations": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "city": {"type": "string"},
                                "state": {"type": "string"}
                            },
                            "required": ["city", "state"]
                        }
                    }
                },
                "required": ["locations"]
            }
        }
    }

    assistant = AssistantAgent(
        name="WeatherReporter",
        system_message="""You are an expert weather reporter.
1. Use the 'get_weather_for_cities' tool to gather detailed data for all cities in the user's request.
2. The tool will return a structured JSON object with current conditions and a 5-day forecast for each city.
3. Present this information in a clear, well-formatted summary. For each city, list the current conditions (temp, wind, alerts) first, followed by the 5-day forecast.""",
        llm_config={
            **llm_config,
            "tools": [correct_tool_definition],
        }
    )

    user_proxy = UserProxyAgent(
        name="UserProxy",
        is_termination_msg=lambda msg: msg.get("tool_calls") is None and msg.get("content", "").strip() != "",
        human_input_mode="NEVER",
        max_consecutive_auto_reply=5,
        code_execution_config=False,
    )

    user_proxy.register_function(
        function_map={"get_weather_for_cities": weather_tool.get_weather_for_cities}
    )

    cities_of_interest = [
        {"city": "St. Louis", "state": "MO"},
        {"city": "Springfield", "state": "IL"},
        {"city": "Indianapolis", "state": "IN"},
        {"city": "Detroit", "state": "MI"},
        {"city": "Columbus", "state": "OH"},
        {"city": "Pittsburgh", "state": "PA"}
    ]

    city_list_str = "\n".join([f"- {c['city']}, {c['state']}" for c in cities_of_interest])
    prompt = f"Please provide a detailed weather report, including current conditions and a 5-day forecast, for the following cities:\n{city_list_str}"
    result = await user_proxy.a_initiate_chat(assistant, message=prompt)
    print("\n--- Final Detailed Report (from within the function) ---")
    print(result.summary)
    return result


task = asyncio.get_running_loop().create_task(run_detailed_weather_report())
final_result = await task

UserProxy (to WeatherReporter):

Please provide a detailed weather report, including current conditions and a 5-day forecast, for the following cities:
- St. Louis, MO
- Springfield, IL
- Indianapolis, IN
- Detroit, MI
- Columbus, OH
- Pittsburgh, PA

--------------------------------------------------------------------------------
WeatherReporter (to UserProxy):

***** Suggested tool call (call_0qwsPBjZ9cGf2vzPEUkhmaBk): get_weather_for_cities *****
Arguments: 
{"locations":[{"city":"St. Louis","state":"MO"},{"city":"Springfield","state":"IL"},{"city":"Indianapolis","state":"IN"},{"city":"Detroit","state":"MI"},{"city":"Columbus","state":"OH"},{"city":"Pittsburgh","state":"PA"}]}
***************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_weather_for_cities...
Call ID: call_0qwsPBjZ9cGf2vzPEUkhmaBk
Input arguments: {'locations': [{'city': 'St. L